# Classificador Regressão Logística - Implantação

Este componente realiza predições usando um modelo de Regressão Logística do [Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html). <br>
Scikit-learn é uma biblioteca open source de machine learning que suporta apredizado supervisionado e não supervisionado. Também provê várias ferramentas para ajustes de modelos, pré-processamento de dados, seleção e avaliação de modelos, além de outras funcionalidades.

Este notebook apresenta:
- como carregar modelos e outros resultados do treinamento.
- como utilizar um modelo para fornecer predições em tempo real.

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [ ]:
%%writefile Model.py
import joblib
import numpy as np
import pandas as pd


class Model(object):
    def __init__(self):
        self.loaded = False

    def load(self):
        # Following links explain why to use load() method insted of __init__()

        # Issue associated with seldon on __init__
        # https://github.com/SeldonIO/seldon-core/issues/2616

        # Solution for the case
        # https://docs.seldon.io/projects/seldon-core/en/latest/python/python_component.html#gunicorn-and-load
        # Carrega artefatos: estimador, etc
        artifacts = joblib.load("/tmp/data/logistic-regression.joblib")
        self.pipeline = artifacts["pipeline"]
        self.features_names_training = artifacts["columns"]
        self.columns_to_filter = artifacts["columns_to_filter"]
        self.label_encoder = artifacts["label_encoder"]
        self.method = artifacts["method"]
        self.new_columns = artifacts["new_columns"]
        self.features_after_pipeline = artifacts["features_after_pipeline"]
        self.loaded = True


    def class_names(self):
        column_names = self._class_names
        return column_names.tolist()

    def predict(self, X, feature_names, meta=None):
        # First time load model
        if not self.loaded:
            self.load()
            
        df = pd.DataFrame(X)
        
        if feature_names:
            # Antes de utilizar o conjunto de dados X no modelo, reordena suas features de acordo com a ordem utilizada no treinamento
            df = pd.DataFrame(X, columns=feature_names)
            X = df[self.columns_to_filter].to_numpy()

        # adiciona a predição ao conjunto de dados
        y_prob = self.pipeline.predict_proba(X)
        y_pred = self.pipeline.predict(X)
        y_pred = self.label_encoder.inverse_transform(y_pred)
        for i in range(y_prob.shape[1]):
            df[self.new_columns[i]] = y_prob[:, i]
        df[self.new_columns[-1]] = y_pred

        self._class_names = df.columns

        return df.to_numpy()